In [21]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import subprocess
import scipy as sc
from itertools import product

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.figsize": (6, 5),
    "text.latex.preamble": "\n".join([
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage[detect-all,locale=FR]{siunitx}",
    ]),
    'lines.markersize': 10,
    'lines.color': 'grey',
    'scatter.marker': '+',
    'errorbar.capsize': 3,
    'savefig.bbox': 'tight',
})

In [22]:
path = '../'
executable = 'bin/ex2'
export_path = path + 'rapport/figures/'
data_path = 'data/'
config_path = 'bin/'
config_name = 'poincarre1'
config_ext = '.conf'

In [23]:
def stringify_dict(d: dict, sep=',') -> str:
    return sep.join(map(lambda a: str(a[0]) + "=" + str(a[1]), tuple(d.items())))

In [24]:
def run(config_file: str, output_file: str, params: dict = {}) -> None:
    options = stringify_dict(params, sep=' ')
    cmd = f"{path}{executable} {path}{config_file} output='{path}{output_file}' {options}"
    print(f"Running command `{cmd}`\n", end='')
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL)
    return params

In [25]:
def load_conf() -> dict:
    conf = {}
    with open(path + config_path + config_name + config_ext, 'r') as f:
        lines = f.read().split('\n')

    for line in lines:
        if not line:
            continue
        name, _, val, *_ = line.split(' ')
        name = name.strip()
        val = val.strip()
        try:
            conf[name] = float(val)
        except ValueError:
            conf[name] = val
    
    return conf

In [26]:
conf = load_conf()

omega0 = np.sqrt(conf['g']/conf['L'])
2*omega0

14.007141035914502

In [27]:
%%bash
cd .. && make clean && make

clang++ -o bin/ex2 src/Ex2_2024_student.cpp -std=c++17 -Wall -Wextra -O2 


In [28]:
from concurrent.futures import ProcessPoolExecutor, Future
all_params = (
    {'theta0': np.pi/6, 'thetadot0': 2},
    *({'theta0': i, 'thetadot0': 0} for i in [
        0.0001, np.pi/6, np.pi/4, 1, np.pi/2, 5*np.pi/6, np.pi-1e-6
    ]),
)

def done(fut: Future):
    print(f"Done processing {fut.result()}")

# Run simulations *IN PARALLEL*
outputs = []  # List to store output file names
with ProcessPoolExecutor(max_workers=8) as p:
    for params in all_params:
        options = stringify_dict(params)
        output_file = f"{data_path}{config_name},{options}.out"
        outputs.append(output_file)
        future = p.submit(run, f'{config_path}{config_name}{config_ext}', output_file, params)
        future.add_done_callback(done)

# outputs = []  # List to store output file names
# for params in all_params:
#     output_file = f"{data_path}{config_name},{stringify_dict(params)}.out"
#     outputs.append(output_file)
#     run(f'{config_path}{config_name}{config_ext}', output_file, params)

Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.7853981633974483,thetadot0=0.out' theta0=0.7853981633974483 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.5235987755982988,thetadot0=0.out' theta0=0.5235987755982988 thetadot0=0`


Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.5235987755982988,thetadot0=2.out' theta0=0.5235987755982988 thetadot0=2`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=1,thetadot0=0.out' theta0=1 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.0001,thetadot0=0.out' theta0=0.0001 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=3.141591653589793,thetadot0=0.out' theta0=3.141591653589793 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=1.5707963267948966,thetadot0=0.out' theta0=1.5707963267948966 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=2.6179938779914944,thetadot0=0.out' theta0=2.6179938779914944 thetadot0=0`
Done processing {'theta0': 0.5235987755982988, 'thetadot0': 2}
Done processing {'theta0': 0.78539816

In [29]:
# Load stuff in memory
all_data = {}

for file, params in zip(outputs, all_params):
    data = np.loadtxt(path + file)
    all_data[file] = (params, np.copy(data))

# Plots!

In [30]:
%matplotlib qt
# plt.close()
for (params, data) in all_data.values():
    theta, thetadot = data[:,1], data[:,2]
    modded = np.mod(theta+np.pi,2*np.pi)-np.pi
    # if params['thetadot0']>0:
    plt.scatter(modded, thetadot, s=1, marker='.', 
                label=f"$\\theta_0={params['theta0']:.2f}, \\dot{{\\theta_0}}={params['thetadot0']:.2f} s^{{-1}}$")
    # else:
    #     plt.scatter(modded, thetadot, s=1, marker='.', label=f"$\\theta_0={params['theta0']:.2f}$")
plt.xlabel(r'$\theta$ [rad]')
plt.ylabel(r'$\dot{\theta}$ [rad/s]')
plt.legend(loc='center left', bbox_to_anchor=(1.1, 0.5), markerscale=5)
# plt.grid()
plt.xlim(-np.pi, np.pi)
ticks = np.arange(-np.pi, np.pi+0.1, np.pi/2)
# labels = [
#     r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$',
#     r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$', r'$\pi$',
# ]
labels = [
    r'$-\pi$', r'$-\frac{\pi}{2}$',
    r'$0$', r'$\frac{\pi}{2}$', r'$\pi$',
]
plt.xticks(ticks, labels)
# plt.axis('equal')
plt.savefig(export_path + 'poincare_overkill.png')
plt.show()